In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms, tv_tensors
import torch.utils.data
from torchvision.transforms import v2
from torchvision.io import read_image
from torch.nn.utils.rnn import pad_sequence
from torchvision.utils import draw_bounding_boxes

import numpy as np
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
plt.ion()   # interactive mode

In [2]:
#from torchvision.io.image import decode_image
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights #fcn_resnet50, FCN_ResNet50_Weights
from torchvision.models.segmentation import lraspp_mobilenet_v3_large, LRASPP_MobileNet_V3_Large_Weights
from torchvision.models.segmentation.lraspp import LRASPPHead
from torchvision.transforms.functional import to_pil_image

In [3]:
import coco_utils, presets, transforms, utils, v2_extras
from torchvision.datasets import wrap_dataset_for_transforms_v2

In [4]:
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights

In [5]:
torch.manual_seed(0)

In [6]:
torch.cuda.is_available()

True

In [7]:
device=torch.device('cuda')
print(f"Using {device} device")

Using cuda device


In [8]:
cudnn.enabled = True

In [9]:
cudnn.benchmark = True

In [10]:
num_classes=4

In [11]:
model = lraspp_mobilenet_v3_large(weights=None,weight_backbone=None)

In [12]:
model.classifier.high_classifier.out_channels= num_classes
model.classifier.low_classifier.out_channels= num_classes

In [13]:
model

LRASPP(
  (backbone): IntermediateLayerGetter(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1

In [14]:
transforms=v2.Compose([v2_extras.CocoDetectionToVOCSegmentation(),
                       #v2.RandomResizedCrop((520), antialias=True),
                       v2.Resize(520),v2.PILToTensor(),
                       #v2.PILToTensor(),
                       v2.ToDtype(torch.float32, scale=True),
                       v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225], inplace=True)])

In [15]:
transforms

Compose(
      CocoDetectionToVOCSegmentation()
      Resize(size=[520], interpolation=InterpolationMode.BILINEAR, antialias=True)
      PILToTensor()
      ToDtype(scale=True)
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=True)
)

In [16]:
train="data/av_sem/train"
train_ann="data/av_sem/train/_annotations.coco.json"

In [17]:
val="data/av_sem/valid"
val_ann="data/av_sem/valid/_annotations.coco.json"

In [18]:
dataset = torchvision.datasets.CocoDetection(train, train_ann, transforms=transforms)
dataset = wrap_dataset_for_transforms_v2(dataset, target_keys={"masks", "labels"})

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


In [19]:
val_ds = datasets.CocoDetection(val,val_ann, transforms=transforms)
val_ds = datasets.wrap_dataset_for_transforms_v2(val_ds, target_keys=["masks","labels"])

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [20]:
train_sampler = torch.utils.data.RandomSampler(dataset)
test_sampler = torch.utils.data.SequentialSampler(val_ds)

In [21]:
dataset.coco.getCatIds()

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [22]:
dataset.coco.cats

{0: {'id': 0, 'name': 'air-force-air-vehicles', 'supercategory': 'none'},
 1: {'id': 1, 'name': 'a-10', 'supercategory': 'air-force-air-vehicles'},
 2: {'id': 2, 'name': 'ac-130', 'supercategory': 'air-force-air-vehicles'},
 3: {'id': 3, 'name': 'b-1', 'supercategory': 'air-force-air-vehicles'},
 4: {'id': 4, 'name': 'b-2', 'supercategory': 'air-force-air-vehicles'},
 5: {'id': 5, 'name': 'b-52', 'supercategory': 'air-force-air-vehicles'},
 6: {'id': 6, 'name': 'f-15', 'supercategory': 'air-force-air-vehicles'},
 7: {'id': 7, 'name': 'f-22', 'supercategory': 'air-force-air-vehicles'},
 8: {'id': 8, 'name': 'f-35', 'supercategory': 'air-force-air-vehicles'}}

In [23]:
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=64,
    num_workers=4,
    #shuffle=True,
    sampler=train_sampler,
    collate_fn= utils.collate_fn#lambda batch: tuple(zip(*batch)) 
)

In [24]:
val_data_loader = torch.utils.data.DataLoader(
    val_ds,
    batch_size=11,
    num_workers=4,
    #shuffle=False,
    sampler=test_sampler,
    collate_fn= utils.collate_fn#lambda batch: tuple(zip(*batch))
)

In [25]:
model = model.to(device)

In [26]:
dl=len(data_loader)

In [27]:
#criterion = nn.CrossEntropyLoss().cuda()
params = [p for p in model.parameters() if p.requires_grad]
# Observe that all parameters are being optimized
optimizer = optim.SGD(params, lr=0.9, momentum=0.9, weight_decay=0.000001)
scaler=torch.amp.GradScaler('cuda',enabled=True)
# Decay LR by a factor of 0.1 every 7 epochs
#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
lr_scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer, total_iters=dl*5, power=0.9)

In [28]:
import train as engine

In [29]:
for epoch in range(5):
    # train for one epoch, printing every 10 iterations
    engine.train_one_epoch(model, engine.criterion, optimizer, data_loader, lr_scheduler, device, epoch, print_freq=1, scaler=scaler)
    # update the learning rate
    #lr_scheduler.step()
    # evaluate on the test dataset
    #engine.evaluate(model, val_data_loader, device, num_classes)

C:\Users\arinz\image analysis\segmentation\lraspp_seg\train.py:110: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch: [0]  [0/6]  eta: 1:42:12  lr: 0.8729544401714396  loss: 3.2633 (3.2633)  time: 1022.0687  data: 57.3004  max mem: 15373
Epoch: [0]  [1/6]  eta: 0:54:32  lr: 0.8458154393298501  loss: 0.8517 (2.0575)  time: 654.4373  data: 28.6738  max mem: 15385
Epoch: [0]  [2/6]  eta: 0:35:15  lr: 0.818579318474666  loss: 0.8517 (1.6304)  time: 528.9890  data: 19.1595  max mem: 15385
Epoch: [0]  [3/6]  eta: 0:23:16  lr: 0.7912421117431193  loss: 0.7761 (1.4064)  time: 465.5129  data: 14.4088  max mem: 15385
Epoch: [0]  [4/6]  eta: 0:14:14  lr: 0.7637995321102683  loss: 0.7761 (1.2502)  time: 427.4243  data: 11.5490  max mem: 15385
Epoch: [0]  [5/6]  eta: 0:06:41  lr: 0.7362469314457727  loss: 0.7347 (1.1507)  time: 401.9493  data: 9.6348  max mem: 15385
Epoch: [0] Total time: 0:40:12
Epoch: [1]  [0/6]  eta: 0:28:46  lr: 0.7085792537238788  loss: 0.5945 (0.5945)  time: 287.6859  data: 13.5867  max mem: 15385
Epoch: [1]  [1/6]  eta: 0:23:25  lr: 0.6807909798625792  loss: 0.5232 (0.5589)  time: 28

In [30]:
torch.save(model,'models/av_lraspp_seg_custom_large.pt')

In [31]:
torch.save(model.state_dict(),'models/av_lraspp_seg_custom_large.pth')

# error corrections